<a href="https://colab.research.google.com/github/ShoSato-047/STAT380/blob/main/ShoSato_Activity_4_(2025)_Likelihoods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Refer to notebooks 3.1 and 3.2

# Introduction

Recall the log-likelihood for a parameter $\theta$ involving a sample $Y_1,...,Y_n$ of *discrete* random response variables with observed values $k_1,...,k_n$:

$$logLik(\theta) = \log\left(\prod_{i=1}^n P(Y_i = k_i; \theta)\right) =  \sum_{i=1}^n \log(P(Y_i = k_i;\theta))$$



Gilovich, Vallone, and Tversky (1985) wrote a controversial but compelling article claiming that there is no such thing as “the hot hand” in basketball. That is, there is no data-based evidence that shooters have stretches where they are more likely to make consecutive shots, and basketball shots are essentially independent events. One of the many ways they tested for evidence of a “hot hand” was to record sequences of shots for players under game conditions and determine if players are more likely to make shots after made baskets than after misses. For instance, assume we recorded data from one player’s first 5 three-point attempts over a 5-game period. B represents a basket, M represents a miss.

Game | First 5 shots |
--- | --- |
1 | BMMBB
2 | MBMBM
3 | MMBBB
4 | BMMMB
5 | MMMMM



We could represent these data in a way that better emphasizes the fact that the observational unit is the ***shot***, not the ***game***, as follows:

In [ ]:
hothand <- read.csv('https://www.dropbox.com/s/wyvu3jtojfepoa8/hothand.csv?dl=1')
hothand

Shot,Game,Y,X
<int>,<int>,<int>,<int>
1,1,1,0
2,1,0,1
3,1,0,0
4,1,1,0
5,1,1,1
6,2,0,0
7,2,1,0
8,2,0,1
9,2,1,0


Here, $Y_i$ is the binary variable measured on each shot, $Y_i=1$ for B and $Y_i=0$ for M.  $X_i$ is a binary variable where $X_i=1$ if the previous shot was a B, and $X_i=0$ otherwise (previous shot was a miss or it was the first shot).



# Question 1: Model 1

Consider the simplest one-parameter model, **Model 1**, which models these data assuming no hot-hand (probability of a basket is the same on each shot).  In this model, $X_i$ is ***not*** related to $Y_i$:


**Model 1: no hot-hand; probability of a basket is the same on each shot (a one-parameter model):**

$$P(Y_i = k_i; p) = p^{k_i}(1-p)^{1-k_i}, k_i \in \{0,1\}, i = 1, 2, ..., 25$$



## A)

Consider the first five shots:

In [ ]:
head(hothand, 5)

,Shot,Game,Y,X
,<int>,<int>,<int>,<int>
1,1,1,1,0
2,2,1,0,1
3,3,1,0,0
4,4,1,1,0
5,5,1,1,1


In [ ]:
# Just plug in Y (1 or 0) into the equation from the above

 What does each of these shots contribute to the Model 1 log-likelihood?

* Shot 1: $log(p^1 (1-p)^{1-1}) = log(p)$
* Shot 2: $log(p^0 (1-p)^{1-0}) = log(1 - p)$
* Shot 3: $log(p^0 (1-p)^{1-0}) = log(1 - p)$
* Shot 4: $log(p^1 (1-p)^{1-1}) = log(p)$
* Shot 5: $log(p^1 (1-p)^{1-1}) = log(p)$

## B)

How many total contributions of $\log(p)$ are there to the log-likelihood?   
  - **There are 10: one for each make.**

How many total contributions of $\log(1-p)$ are there to the log-likelihood?
  - **There are 15: one for each miss.**

In [ ]:
library(dplyr)
(hothand
  %>% summarize(nrows = n(), .by = Y)
)

Y,nrows
<int>,<int>
1,10
0,15


## C)

Complete the function below that calculates the log-likelihood for a given value of $p$.


In [ ]:
logLik.model1 <- function(p) {
  log.lik <- 10*log(p) + 15*log(1-p) #Add all the contributions to the log-likelihood together
  return(log.lik)
}

## D)

Consider 3 different values of $p$: $p = 0.1$; $p = 0.25$; $p = 0.5$.  Use your function to evaluate $logLik(p)$ for each of these.  Which of these $p$ make the observed data most likely?

In [ ]:
logLik.model1(0.1)
logLik.model1(0.25)
logLik.model1(0.4) # This is the best

[1] -24.60626

[1] -18.17817

[1] -16.82529

## E)

Use the code below to find the maximum likelihood estimator, $\hat p_{MLE}$.  What is the MLE, and how does it represent the observed data?

In [ ]:
optimize(logLik.model1, interval = c(0,1), maximum = TRUE)

$maximum
[1] 0.4000015

$objective
[1] -16.82529

In [ ]:
round(optimize(logLik.model1, interval = c(0,1), maximum = TRUE)$maximum,4)

[1] 0.4

In [ ]:
# Notice that 10/25:
10/25

[1] 0.4

# Question 2: Model 2

Now consider **Model 2**.  For Model 2 we want to assume there *is* a hot-hand; that is, the probability of a basket *following a basket* is different from the probability of a basket *any other time*.  This means that we need 2 parameters to fit Model 2: $p_0$, which represents $P(Y_i=1|X_i=0)$ (probability of making a shot following a made shot); and $p_1$, which represents $P(Y_i = 1|X_i = 1)$ (probability of making a shot in all other situations, including the first shot and after a missed shot):


$$P(Y_i = k_i | X_i) = p_0^{k_i(1-X_i)}(1-p_0)^{(1-k_i)(1-X_i)}p_1^{k_iX_i}(1-p_1)^{(1-k_i)X_i}, i = 1, 2, ..., 25$$

This is the same as saying:

* For all data values where $X_i = 1$ (previous shot was a basket):

$$P(Y_i = k_i|X_i = 1) = p_1^{k_i}(1-p_1)^{1-k_i}$$

* For all data values where $X_i = 0$ (previous shot was a miss or it's the first shot of the game):

$$P(Y_i = k_i |X_i = 0) = p_0^{k_i}(1-p_0)^{1-k_i}$$

## A)

Consider the first five shots.


In [ ]:
head(hothand, 5)

,Shot,Game,Y,X
,<int>,<int>,<int>,<int>
1,1,1,1,0
2,2,1,0,1
3,3,1,0,0
4,4,1,1,0
5,5,1,1,1



What does each of these shots contribute to the Model 2 log-likelihood?

* Shot 1: $log(p_0)$
* Shot 2: $log(1-p_1)$
* Shot 3: $log(1-p_0)$
* Shot 4: $log(p_0)$
* Shot 5: $log(p_1)$

## B)

The code below calculates the contribution of each $(X_i, Y_i)$ pair to the Model 2 log-likelihood, when $p_0 = p_1 = 0.5$, then sums the contributions:


In [ ]:
library(dplyr)
p0 <- 0.5
p1 <- 0.5
model2_df <- hothand  %>%
   mutate(model2.loglik.contribution = case_when(X==0 & Y==1~log(p0),
                                                X==0 & Y==0 ~ log(1-p0),
                                                X==1 & Y==1 ~ log(p1),
                                                X==1 & Y==0 ~ log(1-p1)))
head(model2_df)

,Shot,Game,Y,X,model2.loglik.contribution
,<int>,<int>,<int>,<int>,<dbl>
1,1,1,1,0,-0.6931472
2,2,1,0,1,-0.6931472
3,3,1,0,0,-0.6931472
4,4,1,1,0,-0.6931472
5,5,1,1,1,-0.6931472
6,6,2,0,0,-0.6931472


In [ ]:
#The value of the Model 2 log-likelihood when p0 = p1 = 0.5:
(loglik.model2.5050 <- sum(model2_df$model2.loglik.contribution))

[1] -17.32868

Using this code to help you, complete the function below that takes a *vector* of p's containing $(p_0, p_1)$ and returns the log-likelihood.

In [ ]:
loglik.model2 <- function(pvec) {
  p0 <- pvec[1]
  p1 <- pvec[2]
  #Calculate the contributions to the log-likelihood from each data row:
  model2_df <- hothand  %>%
   mutate(model2.loglik.contribution = case_when(X==0 & Y==1~log(p0),
                                                X==0 & Y==0 ~ log(1-p0),
                                                X==1 & Y==1 ~ log(p1),
                                                X==1 & Y==0 ~ log(1-p1)))

  #sum up the n = 25 contributions:
  loglik <- sum(model2_df$model2.loglik.contribution)
  #Return the result:
  return(loglik)
}

In [ ]:
loglik.model2(c(0.1, 0.2))

[1] -22.99795

## C)

Now use `optim()` to find the Model 2 MLEs of $p_0$ and $p_1$.  What are the MLEs, and how do they represent the observed data?  

In [ ]:
#We'll use initial values equal to the Model 1 MLE:
optim(loglik.model2, par = c(0.4, 0.4), control = list(fnscale = -1))

$par
[1] 0.3889118 0.4285329

$value
[1] -16.80883

$counts
function gradient 
      39       NA 

$convergence
[1] 0

$message
NULL

$P_0$ = 0.389 (x = 0)

$P_1$ = 0.429 (x = 1)

In [ ]:
(hothand
  %>% summarize(nrows = n(), .by = c(X, Y))
  %>% mutate(pct = nrows/sum(nrows), .by = X)
  %>% filter(Y == 1)
)

X,Y,nrows,pct
<int>,<int>,<int>,<dbl>
0,1,7,0.3888889
1,1,3,0.4285714


## D)

What do the Model 2 parameter estimates suggest about how $X_i$ changes the probability of making a basket?  Are they *that* different from the Model 1 parameter estimate?

***There is a difference(0.3889 vs. 0.4286). But not sure if the difference is statistically significant or not.***

# Question 3: comparing the models

## A)

Fill in the table below with the relevant quantities allowing us to compare the two models.  Recall:

$$AIC = 2(\mbox{# of parameters})- 2(\mbox{Maximized log-likelihood})$$
$$BIC = \log(n)(\mbox{# of parameters})- 2(\mbox{Maximized log-likelihood})$$

Model | Log-likelihood |  AIC | BIC |
 --- | --- | --- | --- |
 1  | -16.8250 | 35.65  | 36.87 |
 2  | -16.8088 | 37.62 | 40.06 |


Based on these results, which model is preferable?

***Model 2 is preferable by log-likelihood. But, the model 1 is preferable by AIC and BIC.***

In [ ]:
# Model 1 AIC:
-2*-16.825 + 2*1

# Model 1 BIC:
-2*-16.825 + log(25)*1

[1] 35.65

[1] 36.86888

In [ ]:
# Model 2 AIC:
-2*-16.825 + 2*2

# Model 2 BIC:
-2*-16.825 + log(25)*2

[1] 37.65

[1] 40.08775

## B)

We can also use the likelihood ratio test to compare these models.  Recall when testing nested models:


$$H_0: \mbox{Simple model is sufficient}$$
$$H_a: \mbox{Larger model is an improvement}$$


We test these by forming the following test statistic:

$$\Lambda = 2\left(\mbox{maximized log-likelihood(large model)}-\mbox{maximized log-likelihood(simpler model)}\right)$$

A big improvement from fitting the large model will result in a large value of $\Lambda$.  Determining whether the improvement is "significant" can be determined by comparing $\Lambda$ to a $\chi^2$ distribution with  $df=\mbox{# parameters in large model}-\mbox{# parameters in simpler model}$.

Find $\Lambda$ for this problem, and its associated p-value.  What does this result indicate?

In [ ]:
# Difference:
-16.808 --16.825

[1] 0.017

In [ ]:
# Test statistics:
2*(-16.808 --16.825)

[1] 0.034

In [ ]:
# We don't need a larger model (p = 0.85)
1 - pchisq(0.0339999, df = 2-1)

[1] 0.853707

## C)

What do these your results so far suggest about the ability of $X_i$ to model probability of making a basket, i.e., whether there is evidence to support a "hot hand"?

***There is no evidence to support the null hypothesis (large model). In the other words, there is no evidence to support a "hot hand" (p = 0.8537).***